In [ ]:
#necessary modules needed 
#pandas
#numpy
#BioPython

#softwares
#abyss
#Environments
#enviroments prepare
import pandas as pd;import numpy as np;import os;import glob;import re;from datetime import datetime
import sys
from Bio import SeqIO
#import self-made functions
#add path if necessary
#sys.append("/path/to/mut11k_funcstore")
from mut11k_funcstore import *

In [ ]:
#prepare the AB_11k seqs
AB_ref=SeqIO.to_dict(SeqIO.parse("Sly1_11kABref.fa",'fasta'))
#prepare the bam list as bams
bams=[]
#set the parent dir before usage
working_parent_dir=""


In [ ]:
#0 step, preppare the samples, bam list and target_directory
# bams=glob.glob("/mnt/san3/usr/wnn/kohp1-realnbam/kohp1*.realn.bam")
samples=[os.path.basename(bam).replace(".realn.bam","") for bam in bams]
if not os.path.exists(working_parent_dir):os.mkdir(working_parent_dir)
target_dirs=[os.path.join(working_parent_dir,sample) for sample in samples]

In [ ]:

#1st step. Filter reads, output fq and assemble with abyss
for i in range(len(bams)):
    bam_file=bams[i];sample=samples[i];target_dir=target_dirs[i]
    from_bam_to_reads(sample,bam_file,target_dir)

script_file=os.path.join(working_parent_dir,f"run_abyss_{datetime.now().year}_{datetime.now().month}_{datetime.now().day}.sh")
print(script_file)
lines=""
for i in range(len(bams)):
    bam_file=bams[i];name=samples[i];target_dir=target_dirs[i]
    lines+=f"abyss-pe name={name} -C {target_dir} B=1G k=95 in='{target_dir}/{name}_1.fq {target_dir}/{name}_2.fq'\n"
open(script_file,'w').write(lines)
#run the script file in command line


In [ ]:
#2nd step, run the script made 
#bash /mnt/san1/usr/hj/Data/01.neurospora/05.analysis/11k_mut_denovo/denovo_20230718/run_abyss_2025_2_17.sh
# the denovo result will be in the sample directory under parent directory
#3rd step from scaffold to snps
for sample in samples:
    scaffold_align_to_frame(sample,AB_ref,AB_ref,working_parent_dir)